## Aula 8 - Parte 1 - Pesquisar por videos

O primeiro passo é importar a biblioteca apiclient que deve ter sido instalada. Caso não tenha instalado, abra o CMD ou Terminal e digite:

    pip install google-api-python-client
    
Iremos importar uma função específica da biblioteca que irá construir a API.

In [ ]:
from apiclient.discovery import build

Acesse https://console.developers.google.com/apis/credential e recupere a chave de acesso.

In [ ]:
nome_servico = "youtube"
versao = "v3"
chave = ""

Para ter acesso as funcionalidades da API, iremos criar um objeto que contém o serviço a ser utilizado, a versão e a chave gerada anteriormente.

In [ ]:
api = build(nome_servico, versao, developerKey=chave)

O método utilizado para realizar a pesquisa será ```search().list()```. Alguns dos parâmetros possíveis são apresentados abaixo:

_Parâmetros obrigatórios_
- ```part``` especifica uma lista separada por vírgulas de uma ou mais propriedades de recurso ```search``` que serão incluídas pela resposta da API. Os valores que podem ser incluídos são: ```id``` e ```snippet```.

_Parâmetros opcionais_
- ```q``` especifica o termo da consulta a ser pesquisado.
- ```maxResults``` especifica o número máximo e itens que devem ser retornados (máximo 50). **O valor padrão é 5.**
- ```order``` especifica o método que será utilizado para classificar os recursos na resposta da API.
    - ```date``` - são classificados em ordem da data mais atual para a mais antiga.
    - ```rating``` - são classificados da maior para a menor classificação.
    - ```relevance``` - são classificados com base em sua relevância para a consulta da pesquisa. **Este é o valor padrão.**
    - ```title``` - são classificados em ordem alfabética.
- ```type``` - restringe uma consulta de pesquisa para recuperar somente um tipo de recurso. Valores aceitáveis.
    - ```channel``` - Retorna os canais relacionados a busca.
    - ```playlist``` - Retorna as playlists relacionadas a busca.
    - ```video``` - Retorna os videos relacionados a busca.


Outros parâmetros podem ser visualizados em: https://developers.google.com/youtube/v3/docs/search/list?hl=pt-br

Para facilitar, iremos criar uma função que irá recuperar os vídeos com os seguintes parâmetros:

- ```api``` - objeto que contém as informações da API que se deseja utilizar, bem como a credencial de acesso.
- ```consulta``` - termo a ser pesquisado.
- ```quantidade``` - quantos videos queremos recuperar.
- ```part``` - propriedades de recurso que serão incluídas pela resposta da API.

In [ ]:
def pesquisar_videos(api, consulta, quantidade, ordem='relevance', part='id,snippet'):
    resposta = api.search().list(
        q=consulta, 
        maxResults=quantidade, 
        order=ordem, 
        type="video", 
        part=part).execute()
    
    return resposta

In [ ]:
dados = pesquisar_videos(api, 'Copa do Mundo 2018', 50)

In [ ]:
dados.keys()

In [ ]:
dados['pageInfo']

In [ ]:
dados['regionCode']

In [ ]:
len(dados['items'])

In [ ]:
dados['items'][0]

### Estrutura os dados recuperados

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(columns=['videoId', 'title', 'channelTitle', 'channelId', 'description', 'publishedAt', 'url_thumb'])

In [ ]:
for i in range(len(dados['items'])):
    videoId = dados['items'][i]['id']['videoId']
    title = dados['items'][i]['snippet']['title']
    channelTitle = dados['items'][i]['snippet']['channelTitle']
    channelId = dados['items'][i]['snippet']['channelId']
    description = dados['items'][i]['snippet']['description']
    publishedAt = dados['items'][i]['snippet']['publishedAt']
    url_thumb = dados['items'][i]['snippet']['thumbnails']['high']['url']
    df.loc[i] = [videoId, title, channelTitle, channelId, description, publishedAt, url_thumb]

In [ ]:
df

Persistir em disco os dados salvos.

In [ ]:
df.to_csv('50_videos_copa_mundo.csv', sep=';', header=True, index=False, encoding='utf-8')